### **To work with TensorFlow-CPU**
import os</br>
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'</br>
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'</br>

### Import Libraries

In [10]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

**Dataset**

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

* Cement
* Blast furnace slag
* Fly ash
* Water
* Superplasticizer
* Coarse aggregate
* Fine aggregate

**Loading Datasets**

In [11]:
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
df = pd.read_csv(url)
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

**Lets check the shpae of the data**

In [17]:
df.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


**Lets check for any missing value**

In [18]:
df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [19]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


**Lets split the dataset into predictors and target**

Predictors: All columns other than the target column i.e. except predicting column</br>
Target: Column to be predicted

In [28]:
concrete_data_columns = df.columns
predictors = df[concrete_data_columns[concrete_data_columns!='Strength']]
target = df.Strength

We have our predictor and target DataFrame.

In [30]:
print(predictors.head())
print(target.head())

   Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   Coarse Aggregate  Fine Aggregate  Age  
0            1040.0           676.0   28  
1            1055.0           676.0   28  
2             932.0           594.0  270  
3             932.0           594.0  365  
4             978.4           825.5  360  
0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64


Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [34]:
predictors_norm = (predictors-predictors.mean())/predictors.std()
predictors_norm.sample(5)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
529,0.744760,-0.636257,1.356496,-1.290952,0.785983,-0.397651,0.341992,0.163652
242,-0.492485,-0.856472,1.000229,0.638409,-0.017520,0.709688,-0.199318,0.163652
620,-0.259964,-0.856472,-0.846733,0.769530,-1.038638,-0.063263,1.115291,5.055221
33,1.854740,-0.856472,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
466,-0.869496,-0.856472,1.109609,-0.700905,0.618587,1.364317,0.315799,0.860186


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [36]:
n_cols = predictors_norm.shape[1]

**Lets import important `Kera` packages**

In [38]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

**Building a Model**

Lets define a function that can be used for building model.

In [48]:
def regression_model():
    # Build model
    model = Sequential()
    model.add(Input(shape = (n_cols, )))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='mean_squared_error')
    return model

**Train and Test the Model**

In [46]:
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [47]:
model.fit(x=predictors, y=target, validation_split=0.3, epochs=100, verbose=True)

Epoch 1/100
23/23 [==============================] - 0s 8ms/step - loss: 28409.6621 - val_loss: 4991.3560
Epoch 2/100
23/23 [==============================] - 0s 5ms/step - loss: 4807.7671 - val_loss: 5490.9224
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 3030.6311 - val_loss: 3285.4360
Epoch 4/100
23/23 [==============================] - 0s 6ms/step - loss: 2282.4099 - val_loss: 2650.3892
Epoch 5/100
23/23 [==============================] - 0s 5ms/step - loss: 1765.6349 - val_loss: 2088.7034
Epoch 6/100
23/23 [==============================] - 0s 5ms/step - loss: 1341.6141 - val_loss: 1460.5995
Epoch 7/100
23/23 [==============================] - 0s 5ms/step - loss: 1044.5498 - val_loss: 1103.2078
Epoch 8/100
23/23 [==============================] - 0s 5ms/step - loss: 809.9250 - val_loss: 761.1202
Epoch 9/100
23/23 [==============================] - 0s 5ms/step - loss: 663.1348 - val_loss: 606.4716
Epoch 10/100
23/23 [==============================] - 0s 5

**Lets try with 5 number of hidden layers with 50 node each**

In [49]:
def model_building():
    model = Sequential()
    model.add(Input(shape=(n_cols, )))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
model_ = model_building()
model_.fit(predictors, target, validation_split=0.3, verbose=2, epochs=100)

Epoch 1/100
23/23 - 0s - loss: 91176.3281 - val_loss: 16862.4863 - 464ms/epoch - 20ms/step
Epoch 2/100
23/23 - 0s - loss: 7450.4077 - val_loss: 1494.3175 - 130ms/epoch - 6ms/step
Epoch 3/100
23/23 - 0s - loss: 2492.6323 - val_loss: 1578.7469 - 131ms/epoch - 6ms/step
Epoch 4/100
23/23 - 0s - loss: 1758.3322 - val_loss: 1072.7153 - 129ms/epoch - 6ms/step
Epoch 5/100
23/23 - 0s - loss: 1443.4948 - val_loss: 941.4627 - 130ms/epoch - 6ms/step
Epoch 6/100
23/23 - 0s - loss: 1103.1729 - val_loss: 707.9061 - 126ms/epoch - 5ms/step
Epoch 7/100
23/23 - 0s - loss: 596.3918 - val_loss: 373.6565 - 167ms/epoch - 7ms/step
Epoch 8/100
23/23 - 0s - loss: 250.9353 - val_loss: 232.6422 - 120ms/epoch - 5ms/step
Epoch 9/100
23/23 - 0s - loss: 210.7226 - val_loss: 196.4188 - 121ms/epoch - 5ms/step
Epoch 10/100
23/23 - 0s - loss: 191.3893 - val_loss: 179.4727 - 123ms/epoch - 5ms/step
Epoch 11/100
23/23 - 0s - loss: 175.3486 - val_loss: 165.3231 - 123ms/epoch - 5ms/step
Epoch 12/100
23/23 - 0s - loss: 185.565

Based on the results, we notice that:

- Adding more hidden layers to the model increases its capacity to learn and represent complex relationships within the data. This allows the model to better identify, as a result, the model becomes more effective at fitting the training data and potentially improving its predictions.
- By reducing the proportion of data set aside for validation and using a larger portion for training, the model has access to more examples to learn from. This additional training data helps the model improve its understanding of the underlying trends, which can lead to better overall performance.  
